In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/agents-intensive-capstone-project/Hackathon dataset.txt


# Autenticar en el cuaderno

In [2]:
import os
from kaggle_secrets import UserSecretsClient

try:
    GOOGLE_API_KEY = UserSecretsClient().get_secret("GOOGLE_API_KEY")
    os.environ["GOOGLE_API_KEY"] = GOOGLE_API_KEY
    print("✅ Gemini API key setup complete.")
except Exception as e:
    print(
        f"🔑 Authentication Error: Please make sure you have added 'GOOGLE_API_KEY' to your Kaggle secrets. Details: {e}"
    )

✅ Gemini API key setup complete.


#  Importar componentes del ADK

In [3]:
from google.adk.agents import Agent
from google.adk.models.google_llm import Gemini
from google.adk.runners import InMemoryRunner
from google.adk.tools import google_search, AgentTool
from google.genai import types

print("✅ ADK components imported successfully.")

✅ ADK components imported successfully.


# Funciones auxiliares


In [4]:
# Define helper functions that will be reused throughout the notebook

from IPython.core.display import display, HTML
from jupyter_server.serverapp import list_running_servers


# Gets the proxied URL in the Kaggle Notebooks environment
def get_adk_proxy_url():
    PROXY_HOST = "https://kkb-production.jupyter-proxy.kaggle.net"
    ADK_PORT = "8000"

    servers = list(list_running_servers())
    if not servers:
        raise Exception("No running Jupyter servers found.")

    baseURL = servers[0]["base_url"]

    try:
        path_parts = baseURL.split("/")
        kernel = path_parts[2]
        token = path_parts[3]
    except IndexError:
        raise Exception(f"Could not parse kernel/token from base URL: {baseURL}")

    url_prefix = f"/k/{kernel}/{token}/proxy/proxy/{ADK_PORT}"
    url = f"{PROXY_HOST}{url_prefix}"

    styled_html = f"""
    <div style="padding: 15px; border: 2px solid #f0ad4e; border-radius: 8px; background-color: #fef9f0; margin: 20px 0;">
        <div style="font-family: sans-serif; margin-bottom: 12px; color: #333; font-size: 1.1em;">
            <strong>⚠️ IMPORTANT: Action Required</strong>
        </div>
        <div style="font-family: sans-serif; margin-bottom: 15px; color: #333; line-height: 1.5;">
            The ADK web UI is <strong>not running yet</strong>. You must start it in the next cell.
            <ol style="margin-top: 10px; padding-left: 20px;">
                <li style="margin-bottom: 5px;"><strong>Run the next cell</strong> (the one with <code>!adk web ...</code>) to start the ADK web UI.</li>
                <li style="margin-bottom: 5px;">Wait for that cell to show it is "Running" (it will not "complete").</li>
                <li>Once it's running, <strong>return to this button</strong> and click it to open the UI.</li>
            </ol>
            <em style="font-size: 0.9em; color: #555;">(If you click the button before running the next cell, you will get a 500 error.)</em>
        </div>
        <a href='{url}' target='_blank' style="
            display: inline-block; background-color: #1a73e8; color: white; padding: 10px 20px;
            text-decoration: none; border-radius: 25px; font-family: sans-serif; font-weight: 500;
            box-shadow: 0 2px 5px rgba(0,0,0,0.2); transition: all 0.2s ease;">
            Open ADK Web UI (after running cell below) ↗
        </a>
    </div>
    """

    display(HTML(styled_html))

    return url_prefix


print("✅ Helper functions defined.")

✅ Helper functions defined.


# Opciones de reintento

In [5]:
retry_config=types.HttpRetryOptions(
    attempts=5,  # Maximum retry attempts
    exp_base=7,  # Delay multiplier
    initial_delay=1, # Initial delay before first retry (in seconds)
    http_status_codes=[429, 500, 503, 504] # Retry on these HTTP errors
)

# Mi primer agente de IA con ADK

In [6]:
agente_mantenimiento = Agent(
    name="agente_mantenimiento",
    model=Gemini(
        model="gemini-2.5-flash-lite",
        retry_options=retry_config
    ),
    description="Agente especializado en procedimientos de reporte de fallas, reglas del condominio, y horarios de uso de las instalaciones, utilizando el Reglamento Interno como única fuente de normativa.",
    instruction="""
    Eres el Agente de Mantenimiento y Normativa. Tu rol es responder consultas sobre fallas, reparaciones y el reglamento interno del edificio.
    
    Tono: Sé **formal, preciso y directo**. Siempre debes citar el procedimiento o la regla pertinente del 'REGLAMENTO INTERNO' a continuación.
    
    Reglas Clave:
    1.  **Prioriza la seguridad:** Si la consulta es una emergencia (incendio, fuga grave), dirige inmediatamente al usuario a un contacto de emergencia (ej: "Llama al 911 y luego al administrador [número de contacto]").
    2.  **Reporte de Fallas:** Para fallas no urgentes (luz quemada, ascensor lento), pide al usuario que especifique: a) **Ubicación exacta** (piso, área común), b) **Descripción del problema**, c) **Contacto**.
    3.  **Normativa:** Si preguntan sobre ruido, mascotas o uso de instalaciones, proporciona la regla y el horario exacto BASADO EN EL REGLAMENTO.
    
    ---
    
    ## 📚 REGLAMENTO INTERNO (Base de Conocimiento)
    
    REGLAMENTO INTERNO
    REGLAMENTO
    INTERNO
    1

    --- PAGE 2 ---

    REGLAMENTO INTERNO
    NORMAS GENERALES
    1.- ACCESO AL CONDOΜΙΝΙΟ
    a) ACCESO DE VISITAS:
    • El acceso a los edificios deberá ser autorizado por una persona responsable del
    departamento al cual accede la visita.
    •
    Se podrá dar aviso a Conserjería con anterioridad, en cuyo caso el Conserje, una vez
    identificadas las visitas, avisará al departamento de su arribo y luego de eso las dejará
    pasar.
    En caso de no existir aviso previo, el Conserje deberá llamar al departamento para
    avisar de la presencia de la visita y sólo después de ser autorizado por éste la dejará
    pasar.
    Los trabajadores independientes y/o funcionarios de cualquier empresa que preste
    servicios a algún departamento o a la Comunidad, deberán identificarse en Conserjería
    con su cédula de identidad.
    El Conserje deberá anotar en el Libro de Novedades la
    hora de su ingreso, el motivo de la visita y la hora de su salida.
    Es fundamental que el Conserje sepa cuál es el motivo de la visita a fin de controlar
    que estas personas no realicen trabajos que contravengan las disposiciones, tanto de
    éste Reglamento Interno como en el Reglamento de Copropiedad, y que puedan
    afectar la plusvalía del Condominio.
    b) PUERTAS Y PORTONES:
    Las puertas y portones de acceso al recinto de la comunidad, tanto peatonales como
    vehiculares, deberán encontrarse permanentemente cerradas.
    • Será responsabilidad del Conserje de turno cerciorarse de que queden debidamente
    cerradas cada vez que alguien entre o salga del recinto.
    El Personal que en el desempeño de sus funciones deba salir del recinto de la
    Comunidad, deberá hacerlo portando la llave correspondiente a la puerta de acceso y
    asegurando que esta quede debidamente cerrada.
    c) INGRESO DE CORRESPONDENCIA:
    •
    La correspondencia será recibida en Conserjería por lo que el personal externo no
    tendrá acceso al interior del recinto de la Comunidad.
    Lo anterior será válido incluso para la correspondencia Certificada, en cuyo caso, el
    Conserje deberá:

    Llamar al departamento correspondiente a fin de que alguien baje a
    Conserjería a recibir la correspondencia certificada o,
    en caso de ser autorizado por el responsable del departamento, deberá recibir
    la correspondencia, dejar constancia mediante su firma en el acta del
    funcionario de correos y registrar este hecho en el Libro de Novedades de la
    Conserjería.
    La correspondencia deberá ser retirada desde la Conserjería por los residentes de cada
    departamento.
    2

    --- PAGE 3 ---

    REGLAMENTO INTERNO
    d) VENDEDORES:
    No se permitirá el ingreso al recinto de la Comunidad a vendedores, encuestadores,
    promotores, etc., salvo excepción de quienes sean autorizados por algún
    departamento en particular, en cuyo caso esta persona solo tendrá acceso al referido
    lugar.
    • Estas personas deberán identificarse en Conserjería con su cédula de identidad.
    El
    Conserje deberá anotar en el Libro de Novedades la hora de su ingreso, el motivo de la
    visita y la hora de su salida.
    •
    En estas situaciones, el departamento que autorizó la entrada deberá avisar a
    Conserjería al momento de retirarse la visita a fin de evitar que esta acceda
    directamente y sin autorización a otros departamentos.
    e) INGRESO DE OBJETOS PELIGROSOS:
    El Personal deberá estar atento para evitar el ingreso con objetos que puedan producir
    deterioros en los bienes comunes, como ser: artículos pesados que se arrastren,
    objetos con ruedas duras, que puedan ensuciar o deteriorar los pisos o muros, objetos
    voluminosos que entren forzadamente en los ascensores, etc.
    En tales situaciones, el Personal deberá resguardar la integridad de residentes y bienes
    comunes tomando medidas que vayan desde solicitar a la persona que extreme las
    precauciones, hasta prohibir el acceso si no se cuenta con las medidas de seguridad
    adecuadas, pudiendo incluso indicar las vías por las que se podrá circular (caja de
    escalas, ascensores, subterráneo, etc.).
    2.- MUDANZAS
    a) AUTORIZACION:
    • Toda mudanza (de llegada y de salida) deberá ser comunicada a la Administración con
    a lo menos 15 días de anticipación y ser autorizada por ésta previa verificación de que
    los gastos comunes se encuentren debidamente cancelados y al día.
    •
    •
    Para el caso de arrendatarios que abandonan el Condominio, de manera previa a la
    mudanza, el Personal deberá controlar que exista la debida autorización de salida por
    escrito por parte del propietario del departamento.
    De no existir el documento anteriormente citado, el Personal no deberá permitir la
    salida de objetos ni vehículos en proceso de mudanza.
    Durante la mudanza, el Personal de la Comunidad deberá supervisar las labores.
    No está permitido el ingreso de vehículos pesados en mudanzas ya sea de llegada
    como de salida.
    b) DIAS Y HORARIOS:
    Lunes a Viernes de 09.00 a 18.00 horas
    Sábados de 09.00 a 14.00 horas
    Domingos y Feriados no se autoriza por ser días de descanso para toda la comunidad.
    c) GARANTIA:
    La Administración deberá exigir la constitución, de manera previa a la mudanza, de
    una garantía a favor de la Comunidad ascendente a la cantidad de 3 UF.
    3

    --- PAGE 4 ---

    REGLAMENTO INTERNO
    Esta garantía será restituida por la Administración una vez constatado que no se hayan
    producido detrimentos, especialmente en los ascensores, pasillos y cielos de los
    edificios.
    En caso de producirse algún deterioro, la administración estimará el valor de la
    reparación, el cual será descontado de la garantía entregada.
    • De producirse algún reclamo respecto al valor estimado, se esperará a conocer el valor
    definitivo de la reparación antes de devolver el saldo de la garantía.
    Si el valor de reparación supera el monto de la garantía, la diferencia será cargada en
    la próxima cuenta de los gastos comunes del respectivo departamento.
    3.- ESTACIONAMIENTOS
    • Los lugares de estacionamiento se destinarán solamente a estacionar en ellos vehículos de
    propiedad o uso de sus propietarios, no pudiendo arrendarse o cederse a ningún título el
    uso de ellos a personas extrañas o no residentes en el Condominio (Artículo Noveno del
    Reglamento de Copropiedad).
    •
    Los vehículos estacionados deberán posicionarse de manera tal que no obstruyan los
    demás estacionamientos, corredores o pasillos.
    La velocidad máxima permitida para circular por los pasillos de los estacionamientos es de
    $20~Km/hora$
    Solo se les podrá dar la finalidad para lo cual fueron diseñados, por lo que no podrán ser
    utilizados como bodegas.
    El portón de acceso a los estacionamientos cuenta con sistema de tarjeta electrónica que
    permite a cada usuario accionarlo cuando lo necesite, tanto para salir como para entrar al
    Condominio.
    Es obligación de cada usuario portar su respectiva tarjeta para tales efectos.
    Consecuentemente con lo anterior, el Personal tendrá prohibido accionar el portón de
    acceso vehicular.
    Está terminantemente prohibido el uso de la bocina u otro método que produzca
    molestias a la Comunidad para solicitar al Personal la apertura de los portones de acceso a
    los estacionamientos.
    Además, tal como se indica en el párrafo anterior, el Personal no
    tiene autorización para abrir el portón de acceso vehicular.
    En caso de producirse este tipo de situaciones, el Personal deberá estampar la
    correspondiente constancia en el Libro de Novedades y comunicarlo a la Administración.
    Solo se permite al Personal accionar el portón de acceso vehicular en forma manual en
    caso de desperfectos de alguna naturaleza.
    Por razones de seguridad, no se debe permitir a los niños jugar en los sectores destinados
    a estacionamientos (pasillos de circulación y estacionamientos propiamente tales).
    4) BODEGAS
    Están destinadas para guardar en ellas toda clase de de cosas de dominio de sus
    respectivos propietarios, salvo materias corrosivas, inflamables, malolientes o peligrosos
    para la Comunidad (Artículo Noveno del Reglamento de Copropiedad).
    Las bodegas deberán permanecer cerradas con llave, lo que será de exclusiva
    responsabilidad de sus residentes (propietarios o arrendatarios).
    No obstante lo anterior, el Personal de Aseo deberá revisar que las puertas estén cerradas,
    y deberá dejar constancia en el Libro de Novedades de cualquier situación anormal a la vez
    que deberá dar aviso al responsable del departamento correspondiente.
    4

    --- PAGE 5 ---

    REGLAMENTO INTERNO
    5) TELEFONO DE LA COMUNIDAD
    •
    El uso del teléfono estará reservado exclusivamente a los requerimientos para el buen
    funcionamiento de la Comunidad.
    Por lo anterior el Personal no podrá utilizarlo para efectuar llamadas personales (salvo
    emergencias).
    Los residentes podrán, en caso de necesidad, utilizarlo para llamadas de emergencia las
    cuales serán registradas por Conserjería y su importe se cobrará junto con los gastos
    comunes.
    6) ROPA TENDIDA Y OTROS OBJETOS
    Está terminantemente prohibido tender ropa en las barandas de las terrazas o en las
    ventanas de logias, dormitorios u otras.
    De igual manera está prohibido colocar objetos contundentes en las barandas de las
    terrazas, ventanas de las logias y dormitorios y en las ventanas de los espacios comunes
    tales como maceteros u otros (por el alto peligro de lesiones que podrían provocar si se
    llegan a volcar hacia los espacios comunes).
    7) RUIDOS MOLESTOS
    •
    No se podrá emitir ruidos molestos en los espacios comunes así como tampoco en el
    interior de los departamentos.
    En caso de producirse excesos, el Personal deberá llamar por el citófono a una persona
    responsable del departamento causante de los ruidos, solicitando prudencia y
    ponderación.
    De mantenerse o reiterarse la situación, el Personal deberá repetir la notificación
    indicando al responsable del departamento que, de no solucionarse, hará el
    correspondiente reclamo a Carabineros.
    En caso de no solucionarse prontamente el problema, el Personal deberá llamar
    telefónicamente a Carabineros para hacer la denuncia y solicitar su intervención.
    El uso de taladros u otro tipo de herramientas o maquinarias para uso doméstico o de
    trabajo, que originen ruidos molestos, solo se podrá realizar en los siguientes horarios:
    Lunes a Viernes de 09.00 a 18.00 horas
    Ο

    Sábados de 09.00 a 14.00 horas
    o Domingos y Feriados no se autoriza por ser días de descanso para toda la
    comunidad.
    Los equipos e instrumentos musicales deberán funcionar a un volumen que no moleste los
    horarios de descanso de la comunidad detallados anteriormente.
    Los vehículos que ingresen o salgan del Condominio deberán hacerlos con volumen
    adecuado del equipo musical.
    8) COMPORTAMIENTO EN ESPACIOS COMUNES Y ACCESOS
    El Personal deberá velar porque el comportamiento de los residentes y visitas autorizadas
    guarde el debido decoro y sea acorde a las normativas pertinentes en todo momento.
    5

    --- PAGE 6 ---

    REGLAMENTO INTERNO
    9) MASCOTAS
    Está prohibido que las mascotas circulen sin correa por los espacios comunes.
    Está prohibido que las mascotas hagan sus necesidades en los espacios comunes (jardines,
    pasillos u otros).
    En caso de producirse el propietario deberá limpiar en el mismo
    momento.
    10) BASURAS
    Las basuras se deberán eliminar en bolsas plásticas, de tamaño adecuado y bien cerradas
    para evitar suciedad, contaminación y malos olores, a través de los ductos destinados para
    tales fines.
    No se debe eliminar por los ductos: artículos de vidrio, objetos cortantes (botellas, vasos,
    ampolletas, restos de vidrios, etc., etc.), latas, papeles y cartones, palos, fierros,
    escombros u otros artículos de tamaño mayor que se puedan expandir al soltarlos ya que
    pueden obstruir los ductos.
    Estos artículos, si son de tamaño pequeño, deben ser depositados en el cubículo vecino a
    los ductos en cada piso, en el suelo, desde donde serán retirados diariamente por el
    Personal del Condominio (con excepción de los días domingo y feriados).
    Cuando se necesite desechar envases de tamaño grande, como por ej.
    cajas de embalaje
    de televisores, lavadoras, u otros, estos deben ser depositados ordenadamente en la caja
    de escalas del subterráneo por cada residente para evitar que colapsen el cubículo.
    No se deberán botar basuras de ningún tipo en los espacios comunes (colillas de
    cigarrillos, papeles, restos de comidas, etc.).
    No se deberá apagar colillas de cigarrillos en los espacios comunes (jardines, pisos de
    pasillos tanto interiores como exteriores, ascensores, etc.).
    No se deberá sacudir las bolsas de aspiradoras por los ductos ya que contaminan y
    generan molestias a residentes de los pisos inferiores.
    Es obligación de cada residente cuidar nuestro entorno y mantener un medio ambiente
    limpio y sano.
    11) USO DE LOS CARROS (de supermercado)
    •
    Los carros de supermercado deben permanecer en el subterráneo de cada edificio para
    que estén disponibles para quien los necesite.
    El uso de los carros está destinado a transporte de mercaderías de supermercado u otros
    objetos desde los estacionamientos $y/o$ bodegas hacia los departamentos (o viceversa).
    Una vez usados, los carros deberán ser devueltos a su lugar de origen, esto es, a los
    respectivos subterráneos de cada edificio.
    Queda terminantemente prohibido que, una vez usados, los carros sean dejados en los
    estacionamientos, ascensores, pasillos u obstruyendo las puertas de las cajas de escala.
    Por razones de seguridad, los pasillos, los ascensores y las puertas de acceso a las cajas de
    escalas deberán permanecer siempre despejadas.
    Estos son espacios de circulación y,
    como vías de escape, siempre deben encontrarse libres de obstáculos.
    Los carros de supermercado no se deberán utilizar para almacenar escombros, basuras u
    otros materiales de desecho.
    6

    --- PAGE 7 ---

    REGLAMENTO INTERNO
    12) BIENES COMUNES
    •
    •
    Según lo establece el Artículo $5^{\circ}$ del Reglamento de Copropiedad, "son los necesarios para la
    existencia, seguridad y conservación de los edificios y los que permitan a todos y a cada uno de
    los propietarios el uso y goce de la parte que les corresponde, tales como el terreno, los
    cimientos, los muros exteriores y soportantes, la obra gruesa de los suelos, la techumbre, las
    casetas de vigilancia, las instalaciones generales de energía eléctrica, alcantarillado, gas, agua
    potable, recintos de calderas, estanques, etc.".
    Consecuentemente con lo anterior, ningún residente (sea propietario o arrendatario) podrá hacer
    uso para sí de ningún tipo de Bienes Comunes (como por ejemplo uso de fachadas para la
    instalación de antenas de TV, cajas de alarmas, extractores de aire, ampliaciones no autorizadas
    en terrazas, etc.).
    La no observancia de lo anterior, dará origen a multas a beneficio de la Comunidad de hasta
    3 UF, las que se repetirán cada vez que se reiteren las causas que las originen o cada 15 días
    en caso de mantenerse.
    13) SALA MULTIUSO
    El uso de la Sala Multiuso estará reservado paras reuniones familiares a residentes en el
    Condominio.
    Por lo anterior, está terminantemente prohibida la realización de eventos a quienes no vivan en
    el Condominio, reuniones de tipo comercial, política, o celebración de eventos de cualquier
    índole distintos al carácter familiar.
    Su utilización deberá ser solicitada en Conserjería con a lo menos 48 horas de anticipación por
    una persona adulta que sea residente del Condominio.
    El arriendo de la Sala Multiuso tendrá un costo de \$ 5.000 por evento como un aporte por su
    uso.
    Este dinero irá en beneficio de la Comunidad y deberá ser depositado en la cuenta
    corriente de ésta por la administración.
    La sala deberá devolverse en las mismas condiciones en que se la recibió, ordenada y limpia.
    • Para garantizar cualquier deterioro que se pueda producir como consecuencia de la utilización
    de la sala, el residente deberá dejar una garantía equivalente a 3 UF.
    Esta garantía será restituida por la Administración una vez constatado que no se hayan
    producido deterioros como consecuencia de la utilización de la sala.
    En caso de producirse algún deterioro, la administración estimará el valor de la reparación,
    el cual será descontado de la garantía entregada.
    De producirse algún reclamo respecto al
    valor estimado, se esperará a conocer el valor definitivo de la reparación antes de
    devolver el saldo de la garantía.
    Si el valor de reparación supera el monto de la garantía, la diferencia será cargada en la
    próxima cuenta de los gastos comunes del respectivo departamento.
    El uso de la Sala Multiuso estará autorizado con o sin música ambiental en los siguientes
    horarios: Adultos hasta las 03.00 AM, Niños hasta las 23.00 PM.
    En caso de utilizar música ambiental, el volumen deberá ser el adecuado de tal manera
    que no moleste el descanso de los vecinos.
    14) ESTUFAS
    Está prohibido el uso de estufas a parafina en el interior de los departamentos.
    7

    --- PAGE 8 ---

    REGLAMENTO INTERNO
    15) ASCENSORES
    Se deberán llamar sólo al momento que se necesite utilizarlos.
    Se deberá evitar su retención en espera de otras personas.
    Se debe respetar la capacidad máxima de personas indicada en cada uno de los ascensores
    a fin de evitar que colapsen por exceso de peso.
    Está terminantemente prohibido fumar en el interior de los ascensores.
    Por razones de seguridad y buen funcionamiento, no se debe permitir a los niños jugar en
    los ascensores.
    16) BALCONES Y VENTANAS
    Para su limpieza deberán ser trapeadas y no baldeadas, ya que la evacuación de aguas
    provoca molestias a los departamentos de los pisos inferiores.
    Está prohibido sacudir alfombras, manteles o cualquier elemento de uso doméstico hacia
    el exterior.
    Está terminantemente prohibido encender fuego (como por ej. para realizar asados),
    utilizando carbón, madera o cualquier otro elemento combustible en los balcones.
    17) GASTOS COMUNES
    • Los Gastos Comunes deben ser cancelados dentro del plazo indicado por la Administración
    en la correspondiente rendición que se entrega mensualmente a todos los residentes.
    Los gastos Comunes deben ser pagados mediante alguna de las siguientes opciones:
    o Cheque nominativo y cruzado a nombre de la Comunidad.
    Depósito bancario en la cuenta corriente de la Comunidad.
    Ο
    Transferencia electrónica a la cuenta corriente de la Comunidad.
    El cheque o comprobante de pago con cualquiera de las alternativas antes indicadas debe
    ser entregado en Conserjería donde el Personal de turno tiene la obligación de entregar el
    correspondiente recibo de pago debidamente firmado y timbrado.
    Es responsabilidad del residente exigir al Conserje la entrega del recibo de pago cada vez
    que efectúe la cancelación de sus gastos comunes.
    En caso de no exigirse el recibo, la responsabilidad ante el extravío, robo, hurto u otra
    eventualidad será de cada residente (propietario o arrendatario).
    El pago en efectivo será excepcional, por razones que imposibiliten al residente el pago en
    alguna de las formas antes señaladas, y sólo podrá ser cobrado personalmente por el
    Administrador quién hará entrega del recibo correspondiente y depositará en la cuenta
    corriente de la Comunidad el importe de dichos montos.
    El atraso en el pago de los gastos comunes dará origen al cobro de intereses equivalentes
    al 5% del monto total adeudado por el departamento.
    El no pago de gastos comunes durante 3 meses dará origen al corte de energía eléctrica en
    el respectivo departamento.
    18) BICICLETAS Y SIMILARES
    No se podrá circular en bicicletas, patines u otros similares por los pasillos interiores de los
    edificios, ni por los jardines, ni por los estacionamientos, tanto por su seguridad como por
    la seguridad de los demás residentes.
    Las bicicletas o similares no se podrán dejar estacionadas en los espacios comunes
    (jardines, postes de luz, árboles, rejas u otros).
    8

    --- PAGE 9 ---

    REGLAMENTO INTERNO
    Las bicicletas deberán ser estacionadas en el recinto destinado y acondicionado
    especialmente para tales efectos situado a un costado de la plaza de juegos infantiles y
    que no tiene costo para los usuarios.
    19) ASISTENCIA A LAS ASAMBLEAS
    Todo copropietario estará obligado a asistir a las Asambleas respectivas, sea
    personalmente o representado por algún apoderado o por el arrendatario de su
    departamento.
    • La no asistencia a la las Asambleas dará origen a una multa equivalente al valor de 1 UF la
    cual será cargada en los Gastos Comunes el mes siguiente de efectuada la Asamblea.
    Sólo los copropietarios que se encuentran al día en el pago de los gastos comunes, lo que
    se acreditará mediante certificado expedido por el Administrador, podrán optar a cargos
    de representación de la comunidad y concurrir con su voto a los acuerdos que se adopten.
    Cada copropietario tendrá sólo un voto, que será proporcional a sus derechos en los
    bienes de dominio común.
    •
    Los acuerdos adoptados con las mayorías presentes en las Asambleas obligan a todos los
    copropietarios, sea que hayan asistido o no a la sesión respectiva y aun cuando no hayan
    concurrido con su voto favorable a su adopción.
    La asamblea representa legalmente a todos los copropietarios y está facultada para dar
    cumplimiento a dichos acuerdos a través del Administrador y/o del Comité de
    Administración del Condominio.
    Considerando que la Asamblea Ordinaria realizada el jueves 26 de abril de 2012 aprobó
    que para las votaciones que se realicen para dirimir las diferentes opciones que se puedan
    presentar consideren 1 PERSONA = 1 VOTO, las futuras citaciones a Asambleas plantearán
    que cada asistente con uno o más poderes tendrá derecho solo a UN VOTO con el objeto
    de que las decisiones que afectan a la comunidad sean tomadas por los asistentes que son
    los verdaderos interesados y preocupados por los destinos del condominio.
    20) MULTAS
    La no observación de las disposiciones contenidas en la normativa pertinente, entre otras:
    Reglamento de Copropiedad, Reglamento Interno, Ley Sobre Copropiedad Inmobiliaria,
    Reglamento de la Ley Sobre Copropiedad Inmobiliaria, acuerdos de Asambleas de
    Copropietarios, Acuerdos de Comité de Administración, serán objeto de multas a beneficio
    de la Comunidad de hasta 3 UF, las que se repetirán cada vez que se reiteren las causas
    que las originen o cada 15 días en caso de mantenerse.
    Para el cobro de estas multas, la Administración deberá dar aviso al Comité de
    Administración, el que decidirá sobre la procedencia de ésta y el monto a aplicar cada vez
    que sea necesario.
    9

    --- PAGE 10 ---

    REGLAMENTO INTERNO
    DEL PERSONAL
    DEPENDENCIA
    El Personal dependerá directamente del Administrador del Condominio con el cual deberá
    suscribir su contrato individual de trabajo como representante legal de la Comunidad.
    JORNADA DE TRABAJO
    Es el tiempo durante el cual el trabajador debe prestar efectivamente sus servicios a la
    Comunidad en conformidad al contrato de trabajo.
    • La jornada de trabajo ordinaria será de 45 horas semanales.
    •
    •
    •
    El Personal deberá cumplir con exactitud su horario de trabajo (ingreso y retiro del
    Condominio).
    No constituye jornada de trabajo el tiempo que el trabajador permanezca en las
    dependencias de la Comunidad antes o después de la hora de inicio o término de su
    jornada de trabajo.
    No se considerará trabajado el tiempo de colación el cual será de cargo de cada
    trabajador.
    El intervalo de tiempo destinado a colación lo fijará la Administración en función a la
    organización general de las labores a desarrollar y cuya determinación le corresponderá
    en forma exclusiva.
    El trabajador no podrá abandonar el lugar de su trabajo durante la jornada ordinaria o
    extraordinaria sin autorización escrita de su jefe directo.
    Serán consideradas horas extraordinarias las que se trabajen en exceso de la jornada
    pactada, previa autorización y con conocimiento del Administrador.
    Las horas extraordinarias se pagarán con el recargo según lo establece la ley.
    No serán horas extraordinarias las trabajadas en compensación de un permiso, el cual
    deberá ser solicitado por escrito por el trabajador y autorizado por el Administrador.
    Para los efectos de controlar la puntualidad y asistencia del trabajador y determinar las
    horas realizadas, se deberá llevar un registro que consistirá en un Libro de Asistencia del
    personal en el cual deberán registrarse las horas de entrada y salida del Condominio como
    también su entrada y salida de colación.
    PRESENTACION
    Su presentación personal deberá ser óptima, haciendo uso permanentemente de su
    uniforme, manteniéndose siempre peinados, afeitado y ordenado.
    NORMATIVA
    Vigilar que se cumplan permanentemente y fielmente: la Ley Sobre Copropiedad
    Inmobiliaria, el Reglamento de la Ley Sobre Copropiedad Inmobiliaria, el Reglamento de
    Copropiedad, el Reglamento Interno, los acuerdos de Asambleas de Copropietarios y del
    Comité de Administración y todas las instrucciones y normativas que le entregue tanto
    verbalmente como por escrito el Administrador.
    10

    --- PAGE 11 ---

    REGLAMENTO INTERNO 
     
    11 
     
    FUNCIONES
    • 
    Cumplir oportuna y correctamente las instrucciones,  normas y funciones propias 
    de su cargo.
    • 
    Registrar personalmente su asistencia registrando su firma al inicio y término de 
    sus actividades, y también cuando hagan uso del tiempo de colación.
    • 
    Cumplir las medidas que adopte la Administración con el objeto de prevenir robos, 
    hurtos u otros delitos contra la propiedad y contribuir a evitarlos.
    • 
    Cuidará de observar modales respetuosos y comedidos  dentro de la Comunidad, tanto con 
    sus superiores, como con los residentes y compañeros de trabajo.
    • 
    Deberá esmerarse por ser atento, cortés y cooperador con todos los habitantes del 
    Condominio, mostrando un trato igualitario hacia todos los residentes en la Comunidad.
    • 
    Vigilar los diferentes accesos al Condominio, permaneciendo despiertos y atentos en todo 
    el turno que les corresponda.
    • 
    En caso de producirse alguna emergencia, como por ejemplo asaltos, robos, incendio, etc., 
    deberá llamar de inmediato a Carabineros o Bomberos  según sea el caso, y avisará al 
    Administrador.
    • 
    Cuidar las herramientas, maquinarias, instalaciones, elementos de trabajo, útiles, etc., 
    asignados para su uso dentro del trabajo y confiadas a su cuidado.
    • 
    Verificar personal y permanentemente el buen funcionamiento de todos los equipos e 
    instalaciones de la Comunidad (ascensores, bombas de agua, equipo generador eléctrico 
    de emergencia, etc.).
    • 
    Dar cuenta al Administrador de cualquier desperfecto que encuentre en los equipos, 
    maquinarias e instalaciones y en general de cualquier anormalidad que pudiera significar 
    un riesgo para los intereses de la Comunidad.
    • 
    Hacer la reposición de las luminarias quemadas en todos los espacios comunes.
    • 
    Encender o apagar, según sea el caso, el alumbrado eléctrico de los espacios comunes 
    según las necesidades y de acuerdo al inicio y fin de la luz solar.
    • 
    Dar cuenta de las anormalidades al Administrador y de todo lo ocurrido en su turno 
    mediante constancia en el Libro de Novedades de conserjería.
    En caso de ser necesario, 
    dará aviso telefónico al Administrador y requerirá telefónicamente los servicios de las 
    empresas a cargo del mantenimiento del los diferentes equipos e instalaciones de la 
    Comunidad.
    • 
    Debe tomar conocimiento, a través del Libreo de Novedades, de toda las situaciones que 
    pudieren haber ocurrido en su ausencia a fin de poder dar respuestas en caso de que sean 
    requeridas por el Administrador o el Comité de Administración.
    • 
    Regar los jardines con la frecuencia e intensidad que le indicará el Administrador de 
    acuerdo a las necesidades de cada época del año.
    • 
    El Personal de Aseo será el responsable del aseo de  todos los espacios comunes del 
    Condominio.
    • 
    El Mayordomo o Jefe Administrativo tendrá a su cargo la supervisión directa del aseo en 
    todo el Condominio.
    • 
    El resto del personal deberá cumplir funciones de aseo según le indique su jefe directo de 
    acuerdo a las necesidades que se generen en el Condominio.
    • 
    Observar conducta y moralidad intachables.

    --- PAGE 12 ---

    REGLAMENTO INTERNO 
     
    12 
     
    PROHIBICIONES
    • 
    Registrar en el libro correspondiente, la asistencia que no sea la personal de cada 
    trabajador, o hacer marcar la propia por un tercero.
    • 
    No podrá introducir y/o ingerir bebidas alcohólicas  o drogas en su jornada laboral ni en el 
    interior del Condominio aún estando fuera de su jornada de trabajo.
    • 
    No deberá presentarse a trabajar con aliento alcohólico. 
    • 
    Dormir durante sus horas de trabajo.
    • 
    Le queda terminantemente prohibido abandonar el Condominio durante su turno.
    Si fuera 
    absolutamente necesario, dará aviso al Administrador. 
    • 
    Asimismo no podrá salir de compras a solicitud de algún residente del Condominio.
    • 
    No podrá usar herramientas, materiales, útiles, insumos, equipos ni cualquier otro 
    elemento de propiedad de la Comunidad para fines particulares.
    • 
    No podrá realizar trabajos particulares de ninguna naturaleza y bajo ninguna circunstancia 
    durante sus horas de trabajo las cuales deberá dedicar exclusivamente a las necesidades 
    del Condominio.
    • 
    No podrá ingresar a los departamentos durante su jornada de trabajo salvo en casos como 
    las excepciones que más adelante se señalan.
    EXCEPCIONES
    • 
    Durante su jornada laboral solo se le autorizan los  siguientes trabajos o funciones en el 
    interior de algún departamento: accionar los automáticos eléctricos por problemas de 
    cortocircuito (cuando sus residentes no sepan como solucionarlo y a expresa petición de 
    estos), cortar las llaves de paso para el suministro de agua o gas en caso de emergencias u 
    otras instancias o accidentes que, por su gravedad,  requieran de su oportuna intervención.
    • 
    Si estando fuera de su jornada laboral es contratado por algún residente para realizar 
    algún trabajo particular dentro de su departamento,  será de exclusiva responsabilidad de 
    quien contrate sus servicios no siendo responsables  ni la Administración ni el Comité de 
    Administración de cualquier eventual problema posterior.
    """,
    tools=[], # Retiramos la herramienta simulada, el conocimiento está incrustado
)

print("✅ Agente de Mantenimiento actualizado con el Reglamento Interno incrustado en su instrucción.")

✅ Agente de Mantenimiento actualizado con el Reglamento Interno incrustado en su instrucción.


In [7]:
agente_vecindario = Agent(
    name="agente_vecindario",
    model=Gemini(
        model="gemini-2.5-flash-lite",
        retry_options=retry_config
    ),
    description="Agente experto en información local, servicios cercanos, recomendaciones de restaurantes, transporte y puntos de interés del vecindario.",
    instruction="""
    Eres el Agente de Vecindario, el guía de la zona. Tu función es proporcionar información útil y actualizada sobre servicios, comercios, y lugares cercanos.
    
    Tono: Sé **amigable, entusiasta y servicial**.
    
    Reglas Clave:
    1.  **Prioriza la actualidad:** Utiliza obligatoriamente la herramienta de Google Search para obtener direcciones, horarios, reseñas y el estado del transporte en tiempo real.
    2.  **Claridad en las Recomendaciones:** Si recomiendas un lugar (restaurante, tienda), incluye la dirección o una indicación clara de cómo llegar desde el edificio.
    3.  **Variedad:** Ofrece al menos 2-3 opciones cuando el usuario solicite una recomendación (ej: "una farmacia" o "un lugar para comer").
    
    Si el usuario pregunta algo sobre el interior del edificio, indícale amablemente que debe preguntar al 'conserje_bot' o al 'agente_mantenimiento'.
    """,
    tools=[google_search] # Su herramienta clave para datos en tiempo real
)

print("✅ Agente 'agente_vecindario' definido con Google Search.")

✅ Agente 'agente_vecindario' definido con Google Search.


In [8]:
root_agent = Agent(
    name="conserje_bot",
    model=Gemini(
        model="gemini-2.5-flash-lite",
        retry_options=retry_config
    ),
    description="Un agente diseñado para actuar como un conserje virtual y brindar información y asistencia.",
    instruction="Eres Conserje_Bot, el asistente virtual del edificio. Tu tono debe ser cortés, profesional y eficiente. Proporciona información sobre el edificio, servicios locales, y utiliza Google Search para horarios, clima, o datos que requieran ser actuales. Responde siempre con la mayor utilidad posible.",
    tools=[google_search],
)

print("✅ Root Agent defined.")

✅ Root Agent defined.


# Integración de Agentes:

In [9]:
# Asegúrate de que los sub-agentes (definidos previamente) sean envueltos como herramientas
# Los sub-agentes se pasan como herramientas al agente coordinador.
# Se asume que las variables 'agente_mantenimiento' y 'agente_vecindario' ya existen.

root_agent = Agent(
    name="conserje_bot",
    model=Gemini(
        model="gemini-2.5-flash-lite",
        retry_options=retry_config
    ),
    # Esta instrucción es fundamental: le dice al coordinador CÓMO debe enrutar las consultas.
    instruction="""
    Eres Conserje_Bot, el Coordinador de Servicios. Tu objetivo es enrutar las consultas del usuario al agente especializado correcto y consolidar la respuesta.
    
    Reglas de Enrutamiento:
    1.  Si la consulta es sobre **reglas del edificio, reportes de fallas, mantenimiento, o instalaciones internas (ej. piscina, gimnasio)**, debes llamar al `agente_mantenimiento`.
    2.  Si la consulta es sobre **servicios externos, transporte, recomendaciones de restaurantes/tiendas, o puntos de interés local**, debes llamar al `agente_vecindario`.
    3.  Si la consulta no encaja en ninguna categoría (ej. saludos, comentarios triviales), responde directamente.
    
    Debes devolver la respuesta proporcionada por el agente especializado directamente al usuario.
    """,
    # Se envuelven los sub-agentes en 'AgentTool' para que el 'conserje_bot' pueda llamarlos.
    tools=[
        AgentTool(agente_mantenimiento),
        AgentTool(agente_vecindario)
    ],
)

print("✅ Conserje_Bot (Coordinador) creado y agentes unidos.")

✅ Conserje_Bot (Coordinador) creado y agentes unidos.


# Gestionando a conserje_bot

In [10]:
runner = InMemoryRunner(agent=root_agent)

print("✅ Runner created.")

✅ Runner created.


# Consultas:

In [11]:
response = await runner.run_debug(
    "¿cuales son los horarios para mudanzas?"
)


 ### Created new session: debug_session_id

User > ¿cuales son los horarios para mudanzas?


conserje_bot > Lamentablemente, no tengo la información exacta sobre los horarios para mudanzas en este momento. Para obtener detalles precisos y actualizados, te recomiendo consultar el REGLAMENTO INTERNO del condominio, específicamente en la Sección 2, punto b) referente a Días y Horarios para Mudanzas.
